In [235]:
#import face_recognition as fr
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
import glob
import numpy as np
import re
import pandas as pd
import cv2
face_cascade = cv2.CascadeClassifier('gui_2/assets/haarcascade_frontalface_default.xml')
HEIGHT, WIDTH =48,48

In [236]:
paths = ["data/all/train/","data/all/test/"] #["data/CK+/","data/fer/train/"]
# data  = listdir(path)
ignore = ["morralla",".DS_Store","contempt"]
imgs = []
state = []

In [237]:
for path in paths:
    for item in listdir(path):
        if item not in ignore:
            imgs.extend([f"{path}{item}/{p}" for p in listdir(f"{path}{item}")])
            state.extend([item for p in listdir(f"{path}{item}")])

In [241]:
len(imgs)

50964

In [278]:
for path in paths:
    for item in listdir(path):
        if item not in ignore:
            imgs.extend([{"path": f"{path}{item}/{p}", "emotion": item}for p in listdir(f"{path}{item}")])

In [282]:
imgs_ = []
state = []
for p in imgs:
    temp = cv2.imread(p["path"],0)
    faces = face_cascade.detectMultiScale(temp, 1.1, 5)
    if len(faces)==1:
        for (x,y,w,h) in faces:
            recortada = temp[y:y+h, x:x+w]
            recortada = cv2.resize(recortada,(HEIGHT, WIDTH))
            imgs_.append(recortada)
            state.append(p["emotion"])
    
        
imgs_ = [el/255 for el in imgs_]
imgs_array = np.array(imgs_)


In [283]:
imgs_array.shape

(22152, 48, 48)

In [ ]:
errs = []
try:
    imgs_ = []
    for p in imgs:
        temp = Image.open(p)
        save = temp.copy()
        imgs_.append(save)
        temp.close()
except:
    errs.append(p)

In [ ]:
imgs_array = []
HEIGHT, WIDTH =48,48
for f in imgs_:
    img = f.convert("L").resize((HEIGHT, WIDTH))
    imgs_array.append(np.array(img))

In [ ]:
imgs_array = [el/255 for el in imgs_array]
imgs_array = np.array(imgs_array)

In [286]:
imgs_array= imgs_array.reshape((len(imgs_array),48,48,1))

In [288]:
imgs_array.shape

(22152, 48, 48, 1)

In [251]:
state

['surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',

In [289]:
categories = ['sadness', 'happiness', 'anger', 'disgust', 'neutral', 'surprise', 'fear']

In [290]:
categories

['sadness', 'happiness', 'anger', 'disgust', 'neutral', 'surprise', 'fear']

In [291]:
y=pd.DataFrame(state)
y_dummies = pd.get_dummies(y)

In [292]:
y_dummies

,0_anger,0_disgust,0_fear,0_happiness,0_neutral,0_sadness,0_surprise
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...
22147,0,0,0,0,0,1,0
22148,0,0,0,0,0,1,0
22149,0,0,0,0,0,1,0
22150,0,0,0,0,0,1,0


In [293]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs_array,y_dummies)

In [294]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,BatchNormalization,Activation
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from tensorflow.keras.optimizers import Adam, Nadam, SGD, Adamax
import keras

In [295]:
# opt = Adam(learning_rate=0.001)

In [296]:
model = Sequential()
# 1-conv
model.add(Conv2D(64,(3,3),padding='same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 2-conv
model.add(Conv2D(128,(5,5),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 3-conv
model.add(Conv2D(512,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 4-conv
model.add(Conv2D(512,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7,activation='softmax'))

opt = Adam(learning_rate=0.0005)

model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
#model.summary()


In [297]:
from keras.models import Sequential, load_model, model_from_json
from keras import callbacks, optimizers
import tensorflow as tf
from datetime import date

fecha=str(date.today().year)+str(date.today().month)+str(date.today().day)    
symbol = 'alldata'
h5 = symbol + '_2_' + fecha + '_v3.h5'
checkpoint = callbacks.ModelCheckpoint(h5,
                                       monitor='loss',
                                       verbose=0,
                                       save_best_only=True,
                                       #save_weights_only=True,
                                       mode='auto',
                                       save_freq=1)
callback = [checkpoint]
json = symbol + '_best_model' + fecha + '.json'
model_json = model.to_json()
with open(json, "w") as json_file:
    json_file.write(model_json)

modelo = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 5000,callbacks = callback,validation_split = 0.1)

Epoch 1/5000
468/468 [==============================] - 164s 349ms/step - loss: 1.3120 - accuracy: 0.5623 - val_loss: 1.7550 - val_accuracy: 0.5523
Epoch 2/5000
468/468 [==============================] - 7s 15ms/step - loss: 1.0201 - accuracy: 0.6705 - val_loss: 0.8999 - val_accuracy: 0.7034
Epoch 3/5000
468/468 [==============================] - 6s 12ms/step - loss: 0.8980 - accuracy: 0.7010 - val_loss: 0.9983 - val_accuracy: 0.6781
Epoch 4/5000
468/468 [==============================] - 5s 11ms/step - loss: 0.8206 - accuracy: 0.7225 - val_loss: 0.9735 - val_accuracy: 0.7136
Epoch 5/5000
468/468 [==============================] - 6s 13ms/step - loss: 0.7671 - accuracy: 0.7389 - val_loss: 0.8901 - val_accuracy: 0.7262
Epoch 6/5000
468/468 [==============================] - 5s 11ms/step - loss: 0.7236 - accuracy: 0.7483 - val_loss: 0.7286 - val_accuracy: 0.7635
Epoch 7/5000
468/468 [==============================] - 6s 13ms/step - loss: 0.6974 - accuracy: 0.7574 - val_loss: 0.7686 - val

KeyboardInterrupt: 

In [298]:
paths = ["data/predict/"]
# data_pred  = listdir(path)
ignore = ["morralla",".DS_Store","contempt"]
imgs_pred = []
state_pred = []
im_pred = {}

In [299]:
for path in paths:
    for item in listdir(path):
        if item not in ignore:
            imgs_pred.extend([{"path": f"{path}{item}/{p}", "emotion": item}for p in listdir(f"{path}{item}")])
            #state_pred.extend([item for p in listdir(f"{path}{item}")])



In [300]:
len(imgs_pred)

73

In [301]:
state_pred

[]

In [302]:
len(state_pred)

0

In [303]:
imgs_pred_ = []
state_pred = []
for p in imgs_pred:
    temp = cv2.imread(p["path"],0)
    faces = face_cascade.detectMultiScale(temp, 1.1, 5)
    if len(faces)>0:
        for (x,y,w,h) in faces:
            recortada = temp[y:y+h, x:x+w]
            recortada = cv2.resize(recortada,(HEIGHT, WIDTH))
            imgs_pred_.append(recortada)
            state_pred.append(p["emotion"])
    
        
imgs_pred_ = [el/255 for el in imgs_pred_]
imgs_pred_ = np.array(imgs_pred_)
imgs_pred_= imgs_pred_.reshape((len(imgs_pred_),48,48,1))

In [304]:
len(imgs_pred_)

80

In [305]:
len(state_pred)

80

In [306]:
prediction = model.predict(imgs_pred_)

In [315]:
state_pred

['surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'surprise',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'anger',
 'anger',
 'anger',
 'anger',
 'anger',
 'anger',
 'anger',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'sadness',
 'sadness',
 'sadness',
 'sadness',
 'sadness',
 'sadness',
 'sadness']

In [308]:
prediction[:5]

array([[1.0618264e-11, 8.9035246e-12, 5.8411742e-06, 2.6749628e-06,
        4.5739416e-06, 6.9660222e-11, 9.9998701e-01],
       [4.0105770e-06, 8.8373922e-09, 4.1971711e-05, 1.3365221e-03,
        2.6611859e-05, 1.1816149e-09, 9.9859077e-01],
       [3.9991625e-11, 2.0976216e-11, 5.4403685e-04, 6.8589667e-05,
        5.8596584e-06, 6.5326516e-12, 9.9938154e-01],
       [2.4197083e-10, 3.5122205e-12, 8.1186624e-07, 6.7610136e-12,
        4.5277952e-06, 8.9603276e-11, 9.9999464e-01],
       [6.3670960e-08, 3.6881996e-08, 5.6235513e-05, 2.5642853e-06,
        1.8189395e-03, 5.6610801e-08, 9.9812204e-01]], dtype=float32)

In [309]:
prediction.argmax(axis = -1)


array([6, 6, 6, 6, 6, 6, 6, 6, 2, 6, 6, 6, 6, 6, 1, 1, 1, 3, 1, 1, 1, 1,
       1, 1, 1, 3, 1, 1, 1, 4, 1, 1, 1, 4, 4, 1, 0, 0, 0, 0, 4, 4, 4, 4,
       6, 4, 4, 2, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 2, 6, 6, 2,
       6, 6, 2, 2, 2, 4, 6, 5, 5, 4, 5, 5, 5, 5])

In [310]:
cats= y_dummies.columns
cats = [x.replace("0_","") for x in cats]

In [311]:
cats

['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']

In [316]:
len(prediction)

80

In [318]:
states_model = []
i=0
for i in range(len(prediction)):
    states_model.append(cats[prediction[i].argmax()])
    print(state_pred[i],cats[prediction[i].argmax()])

surprise surprise
surprise surprise
surprise surprise
surprise surprise
surprise surprise
surprise surprise
surprise surprise
surprise surprise
surprise fear
surprise surprise
surprise surprise
surprise surprise
surprise surprise
surprise surprise
disgust disgust
disgust disgust
disgust disgust
disgust happiness
disgust disgust
disgust disgust
disgust disgust
disgust disgust
disgust disgust
disgust disgust
disgust disgust
disgust happiness
disgust disgust
disgust disgust
disgust disgust
disgust neutral
disgust disgust
disgust disgust
disgust disgust
anger neutral
anger neutral
anger disgust
anger anger
anger anger
anger anger
anger anger
neutral neutral
neutral neutral
neutral neutral
neutral neutral
neutral surprise
neutral neutral
neutral neutral
neutral fear
neutral neutral
neutral neutral
neutral neutral
happiness happiness
happiness happiness
happiness happiness
happiness happiness
happiness happiness
happiness happiness
happiness happiness
happiness happiness
happiness happiness


In [319]:
from sklearn.metrics import accuracy_score

In [320]:
accuracy_score(state_pred,states_model)

0.7875